<a href="https://colab.research.google.com/github/bongho/Project_ASAC_5th_Review/blob/hoon/%ED%82%A4%EC%9B%8C%EB%93%9C_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B6%9C_%EB%AA%A8%EB%8D%B8_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

import torch
import pandas as pd
import numpy as np

## 리뷰 개별로 테스트

In [3]:
review = pd.read_csv('/content/drive/MyDrive/ASAC 데이터 분석 과정/기업연계_프로젝트/encoded_review_df_filtered.csv')
review.head(1)

df = review

In [3]:
df.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,2348758,1575984,80739,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,3273772,1231666,98537,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
2,5406063,898460,94651,4.0,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07
3,44775,644900,92221,5.0,0,0,0,First time there and it was excellent!!! It fe...,2017-02-19 13:32:05
4,3891834,177785,92778,5.0,0,0,0,"Great burgers,fries and salad! Burgers have a...",2017-01-02 03:17:34


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2680161 entries, 0 to 2680160
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    int64  
 1   user_id      int64  
 2   business_id  int64  
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
dtypes: float64(1), int64(6), object(2)
memory usage: 184.0+ MB


#  ✅ T5- 레스토랑 리뷰 fine-tuning

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("NUSTM/restaurant-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("NUSTM/restaurant-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [13]:
# 입력 텍스트
input_text = '''Cute interior and owner gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today.
Cheese curds were very good and very filling.
Really like that sandwiches come w salad, esp after eating too many curds!
Had the onion, gruyere, tomato sandwich. Wasn't too much cheese which I liked.
Needed something else...pepper jelly maybe. Would like to see more menu options added such as salads w fun cheeses.
Lots of beer and wine as well as limited cocktails.
Next time I will try one of the draft wines'''

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids, num_beams=5, max_length=500, num_return_sequences=4, early_stopping=True)

# 여러 키워드 추출
predicted_keywords = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Keywords:")
for idx, keywords in enumerate(predicted_keywords, 1):
    print(f"{idx}: {keywords}")

Predicted Keywords:
1: beers.
2: bbq.
3: bourbon.
4: beers!


In [55]:
input_text = '''Awesome joint! Today the place was dealing with a short staff in the middle of a blizzard and a lot of ppl coming in, and yet they were hustling a ton, food came out in a normal time and made perfectly.
They were super polite (despite being stressed) and were great about filling the coffee pots as needed too.
Summary: great bagel sandwiches, very good polite service, came through in the clutch.
Very rude staff. The looked miserable.
I had to say excuse me twice to get the girl to look up from her cell phone.
Then she proceeded to give me a look and instruct another miserable employee to help me.
There are plenty of happy people who will be willing to take your job Ordered three bagel sandwiches and a person who ordered ten minutes after me got their food first.
When I asked the cashier about the issue the made a face and said he was working on my order which was a lie because another person who ordered after me received their food before me. Still waiting......oh and the cashier had hickey's all over her neck.
One of the best authentic bagel houses. It is a little on the pricey side but if you want really really delicious bagels, this is the place to go to.
'''

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids, num_beams=3, num_return_sequences=2, early_stopping=True)

# 여러 키워드 추출
predicted_keywords = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Keywords:")
for idx, keywords in enumerate(predicted_keywords, 1):
    print(f"{idx}: {keywords}")

Predicted Keywords:
1: True
2: False report: a blizzard and a short staff


In [28]:
|input_text= '''Straight to the point, it's cheap, it tastes and feels cheap
The good: price, location, dessert (edible food items are: baked chicken, fried chicken, mashed potatoes, Salad bar lol, meat and cheese bar, ravioli, pizza) also our waitress was bomb af
The meh: behind counter meat slicing station, shrimp (not even deveined), Mexican station
The bad: breakfast -worst eggs ever, actually worst, eggs. Le Benedict can't even cut through ham or English muffin (tried on all 3 occasions), steak, shrimp again, hell breaded items can't even, don't even. Decor it's the 1970 nightmare you can imagine
Tried 3 times and failed'''

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids, num_beams=5, num_return_sequences=2, early_stopping=True)

# 여러 키워드 추출
predicted_keywords = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Keywords:")
for idx, keywords in enumerate(predicted_keywords, 1):
    print(f"{idx}: {keywords}")

Predicted Keywords:
1: - - - - - -
2: - - - - - food


In [31]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("NUSTM/restaurant-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("NUSTM/restaurant-t5-base")


input_text = '''If you decide to eat here,
just be aware it is going to take about 2 hours from beginning to end.
We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience.
The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant.
We have just had too many experiences where we spent way too long waiting.
We usually opt for another diner or restaurant on the weekends, in order to be done quicker'''

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids, num_beams=10, max_length=50, num_return_sequences=8, early_stopping=True)

# 여러 키워드 추출
predicted_keywords = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

print("Predicted Keywords:")
for idx, keywords in enumerate(predicted_keywords, 1):
    print(f"{idx}: {keywords}")

Predicted Keywords:
1: :)
2:  service. :)
3:  food time.
4:  service.
5:   service. :)
6:  food  :)
7:  service. :-)
8:   service.


# ✅ Keybert

- distilbert -base

In [5]:
# keyBERT 설치
!pip install keybert

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [6]:
from keybert import KeyBERT

# 모델 로드
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

# 입력 텍스트
input_text = '''If you decide to eat here,
just be aware it is going to take about 2 hours from beginning to end.
We have tried it multiple times, because I want to like it! I have been to its other locations in NJ and never had a bad experience.
The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant.
We have just had too many experiences where we spent way too long waiting.
We usually opt for another diner or restaurant on the weekends, in order to be done quicker'''

# 키워드 추출
keywords = kw_model.extract_keywords(input_text, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=10)

print("Extracted Keywords:")
for keyword in keywords:
    print(keyword)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Extracted Keywords:
('the weekends', 0.2105)
('long waiting', 0.1729)
('another diner', 0.165)
('weekends', 0.1353)
('weekends in', 0.1219)
('the waitstaff', 0.11)
('diner', 0.0889)
('the food', 0.0786)
('many experiences', 0.0713)
('about hours', 0.0682)


In [22]:
input_text = '''Cute interior and owner gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today.
Cheese curds were very good and very filling.
Really like that sandwiches come w salad, esp after eating too many curds!
Had the onion, gruyere, tomato sandwich. Wasn't too much cheese which I liked.
Needed something else...pepper jelly maybe. Would like to see more menu options added such as salads w fun cheeses.
Lots of beer and wine as well as limited cocktails.
Next time I will try one of the draft wines'''

# 키워드 추출
keywords = kw_model.extract_keywords(input_text, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=10)

print("Extracted Keywords:")
for keyword in keywords:
    print(keyword)

Extracted Keywords:
('salads fun', 0.4462)
('fun cheeses', 0.4353)
('tomato sandwich', 0.4254)
('pepper jelly', 0.346)
('cheese curds', 0.3366)
('sandwiches come', 0.2699)
('much cheese', 0.265)
('jelly maybe', 0.2649)
('gruyere tomato', 0.2449)
('eating too', 0.2359)


In [30]:
input_text = '''Straight to the point, it's cheap, it tastes and feels cheap
The good: price, location, dessert (edible food items are: baked chicken, fried chicken, mashed potatoes, Salad bar lol, meat and cheese bar, ravioli, pizza) also our waitress was bomb af
The meh: behind counter meat slicing station, shrimp (not even deveined), Mexican station
The bad: breakfast -worst eggs ever, actually worst, eggs. Le Benedict can't even cut through ham or English muffin (tried on all 3 occasions), steak, shrimp again, hell breaded items can't even, don't even. Decor it's the 1970 nightmare you can imagine
Tried 3 times and failed'''

keywords = kw_model.extract_keywords(input_text, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=10)
print("Extracted Keywords:")
for keyword in keywords:
    print(keyword)

Extracted Keywords:
('steak shrimp', 0.57)
('fried chicken', 0.5199)
('baked chicken', 0.5066)
('ravioli pizza', 0.5)
('chicken fried', 0.4622)
('potatoes salad', 0.4534)
('mashed potatoes', 0.4313)
('worst eggs', 0.4311)
('chicken mashed', 0.4119)
('station shrimp', 0.3948)


### 문장 단위로 뽑아보기

In [16]:

from keybert import KeyBERT

# 모델 로드
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

# 입력 텍스트
input_text = '''If you decide to eat here,
just be aware it is going to take about 2 hours from beginning to end.'''

# We have tried it multiple times, because I want to like it! I have been to its other locations in NJ and never had a bad experience.
# The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant.
# We have just had too many experiences where we spent way too long waiting.
# We usually opt for another diner or restaurant on the weekends, in order to be done quicker'''

# 키워드 추출
keywords = kw_model.extract_keywords(input_text, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=10)

print("Extracted Keywords:")
for keyword in keywords:
    print(keyword)


Extracted Keywords:
('about hours', 0.666)
('hours from', 0.6078)
('hours', 0.6053)
('to eat', 0.4309)
('to end', 0.3944)
('eat', 0.3919)
('is going', 0.3837)
('you decide', 0.3695)
('end', 0.3541)
('going to', 0.3536)


In [18]:
from keybert import KeyBERT

# 모델 로드
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

# 입력 텍스트
# input_text = '''If you decide to eat here,
# just be aware it is going to take about 2 hours from beginning to end.'''

input_text = 'We have tried it multiple times, because I want to like it!'
# I have been to its other locations in NJ and never had a bad experience.'
# The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant.
# We have just had too many experiences where we spent way too long waiting.
# We usually opt for another diner or restaurant on the weekends, in order to be done quicker'''

# 키워드 추출
keywords = kw_model.extract_keywords(input_text, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=10)

print("Extracted Keywords:")
for keyword in keywords:
    print(keyword)


Extracted Keywords:
('multiple times', 0.518)
('have tried', 0.4937)
('tried it', 0.4665)
('it multiple', 0.4385)
('to like', 0.4322)
('want to', 0.4222)
('tried', 0.4208)
('multiple', 0.4069)
('because want', 0.4011)
('want', 0.3877)


문장 단위로 키워드를 추출하면 대표하는 단어를 잘 만들어준다.

In [7]:
from keybert import KeyBERT

# 모델 로드
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

# 입력 텍스트
# input_text = '''If you decide to eat here,
# just be aware it is going to take about 2 hours from beginning to end.'''

#input_text = 'We have tried it multiple times, because I want to like it!'
input_text = 'I have been to its other locations in NJ and never had a bad experience.'
# The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant.
# We have just had too many experiences where we spent way too long waiting.
# We usually opt for another diner or restaurant on the weekends, in order to be done quicker'''

# 키워드 추출
keywords = kw_model.extract_keywords(input_text, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=10)

print("Extracted Keywords:")
for keyword in keywords:
    print(keyword)

Extracted Keywords:
('other locations', 0.3813)
('in nj', 0.3295)
('nj', 0.3112)
('nj and', 0.2805)
('its other', 0.2119)
('never had', 0.1765)
('experience', 0.1718)
('and never', 0.162)
('been to', 0.1578)
('locations', 0.1567)
